In [108]:
import time
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset
from IPython.display import display, clear_output
from torch.autograd import Variable

In [109]:
def status(batch_size, ep, epoch, i, loss, acc, data_loader):
    # status
    clear_output(wait=True)
    print(str(ep) + '/' + str(epoch))
    print('batch: ' + str(i+1) + '/' + str(len(data_loader)) + 
             ' [' + '='*int((i+1)/(len(data_loader)/20)) +
              '>' + ' '*(20 - int((i+1)/(len(data_loader)/20))) +
              ']')
    print('Loss: %.4g   Accuracy: %.4g' % ((loss / ((i+1)*batch_size)),
                                            (acc / ((i+1)*batch_size))))

In [134]:
class PezzGen(nn.Module):
    def __init__(self):
        super().__init__()
        #self.dimIn = dimIn
        #self.dimOut = dimOut
        #self.filterNum = filterNum
        #actFun = nn.LeakyReLU(0.2, inplace=True)
        
        print("\n------Initializing PezzGen------\n")
        
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.BNorm = nn.BatchNorm2d(6)
        self.actFun = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2,2)
        
        self.bridge = nn.Conv2d(6, 6, 3)
        self.BNorm = nn.BatchNorm2d(6)
        
        self.inv1 = nn.ConvTranspose2d(6, 3, 5)
        self.BNorm = nn.BatchNorm2d(2,2)
        self.actFun = nn.ReLU()
        self.up1 = nn.Conv2d(3, 3, 3)
        self.BNorm = nn.BatchNorm2d(3)
        
    def forward(self, x):
        x = self.pool1(self.actFun(self.BNorm(self.conv1(x))))
        x = self.BNorm(self.bridge(x))
        x = self.actFun(self.BNorm(self.inv1(x)))
        x = self.BNorm(self.up1(x))
        return x


net = PezzGen()
              


------Initializing PezzGen------



In [135]:
# parameters
batch_size = 50
img_size = 256
lr = 0.1
epoch = 100

# Generator
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
generator = nn.DataParallel(net)

img_dir = "./maps/"
trainset = dset.ImageFolder(root=img_dir,
                            transform = transforms.Compose([
                            transforms.Scale(size=img_size),
                            transforms.CenterCrop(size=(img_size,
                            img_size*2)),
                            transforms.ToTensor(),
                            ]))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, 
                                          num_workers=2)

recon_loss_func = nn.MSELoss()
gen_optimizer = torch.optim.Adam(net.parameters(),lr=lr)




In [136]:
for ep in range(epoch):
    for i, (image, label) in enumerate(trainloader):
        
        satel_image, map_image = torch.chunk(image, chunks=2, dim=3)
        
        gen_optimizer.zero_grad()
        
        # print data
        #im = im[0,:,:,:]
        #print(image.shape)
        #plt.imshow(np.transpose(im, (1, 2, 0)))
        #plt.show()
        #im = satel_image[0,:,:,:]
        #plt.imshow(np.transpose(im, (1, 2, 0)))
        #plt.show()
        
        #time.sleep(2)
        
        x = Variable(satel_image)
        y_ = Variable(map_image)
        y = generator.forward(x)
        
        current_loss = recon_loss_func(y,y_)
        loss.backward()
        gen_optimizer.step()
        current_acc = np.array(1)
        
        # status
        status(batch_size, ep+1 , epoch, i, current_loss,
               current_acc, trainloader)
        

RuntimeError: running_mean should contain 6 elements not 3

In [80]:
print(enumerate(trainloader, 0))

ValueError: too many values to unpack (expected 2)